# Thieulot et al. 2014 (3D version)

version 0.1

Romain Beucher (romain.beucher@unimelb.edu.au)

The following is an example of a lithospheric scale model.

<img src="data/Thieulot2014.png">

In [ ]:
from __future__ import print_function
import underworld as uw
import underworld.function as fn
import numpy as np

# LMR utilities
from unsupported.LMR import *
from UWGeodynamics.scaling import *
from UWGeodynamics.scaling import nonDimensionalize as nd
from UWGeodynamics.lithopress import lithoPressure
from unsupported.LecodeIsostasy import lecode_tools_isostasy
import unsupported.rheology as rheology

# Output
import h5py
import os

# Visualisation
import glucifer
from underworld.utils import is_kernel
import matplotlib.pyplot as plt

print(uw.__version__)
print(uw.__file__)

In [ ]:
outputDir = './outputs/'
        
if uw.rank() == 0:
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

## Options

In [ ]:
Erosion = False
Sedimentation = True
BottomStressBC = False
BottomLecodeIsostasyBC = True

# Scaling

We define a set of physical values that are chosen to represent some characterics of the system we are aiming to model. Those values are then used to calculate a set of scaling coefficients that we will used to scale the different dimensions of our problem. That step is not mandatory but reduce the potential for numerical errors and is generally considered good practice. The *scaling* python dictionary can be seen as a bridge between real world and model world dimensions.

All the parameters are defined with natural units using the pint python module. There is no obligation to use SI units but that is of course encouraged...As pint knows about prefixes, you can freely use them without worrying about conversion.

In [ ]:
# Characteristic values of the system
half_rate = (0.5 * u.centimeter / u.year).to(u.meter / u.second)
model_length = 192e3 * u.meter
model_width = 64e3 * u.meter
model_height = 50e3 * u.meter
refViscosity = (1e24 * u.pascal * u.second).to_base_units()
surfaceTemp = (0. * u.degC).to_base_units()
baseModelTemp = (550. * u.degC).to_base_units()
bodyforce = (2800 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2)


KL_meters = model_length
KT_seconds = KL_meters / half_rate
KM_kilograms = bodyforce * KL_meters**2 * KT_seconds**2
Kt_degrees = (baseModelTemp - surfaceTemp)
K_substance = 1. * u.mole

scaling = {"[time]": KT_seconds,
           "[length]": KL_meters, 
           "[mass]": KM_kilograms, 
           "[temperature]": Kt_degrees,
           "[substance]": K_substance}

# General parameters

We first define some general parameters and scale them by using the *nonDimensionalize* function available from the *scaling* module. Note that we aliased the function to *nd* in order to facilitate reading but those are equivalent. We will only use *nd* in the following.

In [ ]:
gravity = nonDimensionalize(9.81 * u.meter / u.second**2, scaling)
R = nonDimensionalize(8.3144621 * u.joule / u.mole / u.degK, scaling)

# Geometry of the model

We define the dimensions and extent of the model on a regular cartesian mesh.

In [ ]:
nx = 96
ny = 44
nz = 14

minX = nd(0. * u.kilometer, scaling)
maxX = nd(192. * u.kilometer, scaling)
minY = nd(0. * u.kilometer, scaling)
maxY = nd(64. * u.kilometer, scaling)
minZ = nd(-28. * u.kilometer, scaling)
maxZ = nd(7. * u.kilometer, scaling)

mesh = uw.mesh.FeMesh_Cartesian(elementType = ("Q1/dQ0"),
                                elementRes  = (nx-1, ny-1, nz-1),
                                minCoord    = (minX, minY, minZ),
                                maxCoord    = (maxX, maxY, maxZ))

In our problem, the material history is attached to the particles while the velocity field and the state variables such as the pressure and the temperature are attached to the mesh.
We first need to define a swarm of particles and a set of mesh and swarm variables.

### Particle Swarm

In [ ]:
swarm  = uw.swarm.Swarm( mesh = mesh, particleEscape=True)
swarmLayout = uw.swarm.layouts.GlobalSpaceFillerLayout( swarm = swarm, particlesPerCell=25 )
swarm.populate_using_layout( layout = swarmLayout )

### Mesh Variables

In [ ]:
velocityField = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=mesh.dim )
pressureField = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
solverPressure   = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
temperatureField    = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=1 )
temperatureDotField = uw.mesh.MeshVariable( mesh=mesh ,nodeDofCount=1)
stressField = uw.mesh.MeshVariable( mesh=mesh ,nodeDofCount=3)

# It is considered good practice to initialise the data arrays to 0.
velocityField.data[...]  = 0.0
pressureField.data[...]  = 0.0
solverPressure.data[...]  = 0.0
temperatureField.data[...] = 0.0
temperatureDotField.data[...] = 0.0
stressField.data[...] = 0.0

### Swarm Variables

In [ ]:
materialIndexField = swarm.add_variable( dataType="int", count=1 )
viscosityField  = swarm.add_variable( dataType="double", count=1)
cumulativeTotalStrain = swarm.add_variable( dataType="double", count=1)
strainRate = swarm.add_variable( dataType="double", count=1)
viscosityVar = swarm.add_variable(dataType="double", count=1)

cumulativeTotalStrain.data[...] = 0.0

# Visualisation

Underworld2 offers many options for visualisation. In the context of this Jupyer notebook we will use glucifer functions to visualise the progression of our setup and the results of the model.

In [ ]:
# Pressure Field 
if is_kernel():
    figPressure = glucifer.Figure( figsize=(1200,400))
    figPressure.append(glucifer.objects.Surface(mesh, pressureField))

# Temperature Field
if is_kernel():
    figTemp = glucifer.Figure(figsize=(1200,400))
    figTemp.append(glucifer.objects.Surface(mesh, temperatureField))

# Velocity Field
if is_kernel():
    FigVelocity = glucifer.Figure(figsize=(1200,400))
    FigVelocity.append(glucifer.objects.Points(swarm, materialIndexField, pointSize=3.0))
    FigVelocity.append(glucifer.objects.VectorArrows(mesh, velocityField, scaling=0.03, arrowHead=10., resolutionI=25, resolutionJ=10)) 

# Material Swarm
if is_kernel():
    FigMaterials = glucifer.Figure( figsize=(1200,400))
    FigMaterials.append(glucifer.objects.Points(swarm, fn_colour=materialIndexField, fn_size=2.0))
    
# Cumulative Plastic Strain
if is_kernel():
    FigPlasticStrain = glucifer.Figure( figsize=(1200,400))
    FigPlasticStrain.append(glucifer.objects.Points(swarm, fn_colour=cumulativeTotalStrain, fn_size=2.0))

# Initial Setup

## Material distribution

The way we choose do define the initial set up is based on the distribution of the different materials (or phases).

In [ ]:
air = Material()
crust = Material()
sediment = Material()

materials = [air, crust]

The initial set up is just a layered cake. We define the top and bottom of each layer and use the utils.layer function to define the shape of the material.

In [ ]:
air.top           = maxY
air.bottom        = nd(  0.  * u.kilometer, scaling)
crust.top    = nd(  0.  * u.kilometer, scaling)
crust.bottom = minY

coord = fn.input()
conditions = [((coord[2] < obj.top) & (coord[2] > obj.bottom), obj.index) for obj in materials] 
materialIndexField.data[:] = fn.branching.conditional( conditions ).evaluate(swarm)

In [ ]:
materials.append(sediment)

It is a good idea to check the setup before proceeding any further:

In [ ]:
checkpoint_number = 0
time_years = 0

sH = swarm.save(os.path.join(uw_output_path, 'swarm-%s.h5' % checkpoint_number), scaling, units=u.kilometers)
file_prefix = os.path.join(uw_output_path, 'material-%s' % checkpoint_number)
handle = materialIndexField.save('%s.h5' % file_prefix)
materialIndexField.xdmf('%s.xdmf' % file_prefix, handle, 'material', sH, 'swarm', modeltime=time_years)

## Material properties

The swarm is initialialised but we now need to define the physical properties of our materials:

### Densities

In [ ]:
# Densities
air.density = nd(1. * u.kilogram / u.metre**3, scaling)
crust.density = nd(2800. * u.kilogram / u.metre**3, scaling)
sediment.density = crust.density

### Thermal Diffusivities

In [ ]:
global_diffusivity = nd(1e-6 * u.metre**2 / u.second, scaling) 
for material in materials:
    material.diffusivity = global_diffusivity

### Thermal Capacities

In [ ]:
global_cp = nd(803.57 * u.joule / (u.kelvin * u.kilogram), scaling)
for material in materials:
    material.capacity = global_cp
air.capacity = nd(100. * u.joule / (u.kelvin * u.kilogram), scaling)

### Thermal Expansivities

In [ ]:
global_expansivity = nd(2.5e-5 / u.kelvin, scaling)
for material in materials:
    material.thermalExpansivity = global_expansivity
air.thermalExpansivity = 0.0

### Radiogenic Heat Production

In [ ]:
air.radiogenicHeatProd        = 0.0
sediment.radiogenicHeatProd   = nd(0.9 * u.microwatt / u.meter**3, scaling)
crust.radiogenicHeatProd = nd(0.9 * u.microwatt / u.meter**3, scaling)

### Buoyancy Force

Define how the material densities interact with the gravity field.

In [ ]:
Tref = nd(273.15 *u.degK, scaling)

densityMap = {}
for material in materials:
    densityMap[material.index] = material.density * (1.0 - material.thermalExpansivity * (temperatureField - Tref))
    
densityFn = fn.branching.map( fn_key=materialIndexField, mapping=densityMap )

z_hat = ( 0.0, 0.0, -1.0 )
buoyancyFn = densityFn * z_hat * gravity

## Material Rheologies

In [ ]:
air.rheology = Rheology()
sediment.rheology = Rheology()
crust.rheology = Rheology()

### Strain Rate and Temperature Dependent Viscosity function


In [ ]:
# symmetric component of the gradient of the flow velocity.
strainRateFn = fn.tensor.symmetric( velocityField.fn_gradient )
strainRate_2ndInvariantFn = fn.tensor.second_invariant(strainRateFn) 

In [ ]:
def powerLaw(A, Q, n, Va, R, strainRateFn, pressureFn, temperatureFn, DefaultSRInvariant, f=1.0):
    I = fn.misc.max(fn.tensor.second_invariant(strainRateFn), DefaultSRInvariant)
    P = pressureFn
    T = temperatureFn       
        
    return (f * A**(-1.0 / n) * I**((1.0-n)/n) * 
            fn.math.exp((Q + P * Va)/(R*T*n)))

In [ ]:
SRT_WetQ_Tullis2002 = rheology.SRT.SRT_WetQ_Tullis2002
SRT_WetQ_Tullis2002 = {k: nd(v, scaling) for k, v in SRT_WetQ_Tullis2002.items()}

In [ ]:
  # Define Effective viscosity functions
SRT_WetQ_Tullis2002Fn = powerLaw(R=R,strainRateFn=strainRateFn, pressureFn=pressureField,
                                    temperatureFn=temperatureField, **SRT_WetQ_Tullis2002)

In [ ]:
# Assign function to materials
air.rheology.viscosity        = fn.misc.constant(nd(1e19 * u.pascal * u.second, scaling))
crust.rheology.viscosity = SRT_WetQ_Tullis2002Fn
sediment.rheology.viscosity   = SRT_WetQ_Tullis2002Fn

## Plasticity

In [ ]:
def cohesionWeakening(cumulativeTotalStrain, Cohesion, CohesionSw, epsilon1=0.5, epsilon2=1.5, **kwargs):

    cohesionVal = [(cumulativeTotalStrain < epsilon1, fn.misc.constant(Cohesion)),
                   (cumulativeTotalStrain > epsilon2, fn.misc.constant(CohesionSw)),
                   (True, Cohesion + ((Cohesion - CohesionSw)/(epsilon1 - epsilon2)) * (cumulativeTotalStrain - epsilon1))]

    return fn.branching.conditional(cohesionVal)

def frictionWeakening(cumulativeTotalStrain, FrictionCoef, FrictionCoefSw, epsilon1=0.5, epsilon2=1.5, **kwargs):

    frictionVal = [(cumulativeTotalStrain < epsilon1, fn.misc.constant(FrictionCoef)),
                   (cumulativeTotalStrain > epsilon2, fn.misc.constant(FrictionCoefSw)),
                   (True, FrictionCoef + ((FrictionCoef - FrictionCoefSw)/(epsilon1 - epsilon2)) * (cumulativeTotalStrain - epsilon1))]

    frictionVal = fn.branching.conditional(frictionVal)
    
    return fn.math.atan(frictionVal)

In [ ]:
Yield_HuismansBeaumont2007 = {"Cohesion": 20. * u.megapascal,
                        "CohesionSw": 10. * u.megapascal,
                        "FrictionCoef": 0.268,
                        "FrictionCoefSw": 0.035,
                        "MinimumViscosity": 1e19 * u.pascal * u.second}

In [ ]:
Yield_HuismansBeaumont2007 = {k: nd(v, scaling) for k, v in Yield_HuismansBeaumont2007.items()}

In [ ]:
sediment.rheology.cohesion   = cohesionWeakening(cumulativeTotalStrain, epsilon1=0.25, epsilon2=1.25, **Yield_HuismansBeaumont2007)
crust.rheology.cohesion = cohesionWeakening(cumulativeTotalStrain, epsilon1=0.25, epsilon2=1.25, **Yield_HuismansBeaumont2007)

In [ ]:
sediment.rheology.friction   = frictionWeakening(cumulativeTotalStrain, epsilon1=0.25, epsilon2=1.25, **Yield_HuismansBeaumont2007)
crust.rheology.friction = frictionWeakening(cumulativeTotalStrain, epsilon1=0.25, epsilon2=1.25, **Yield_HuismansBeaumont2007)

In [ ]:
#Plane Strain Drucker-Prager
DefaultSRInvariant = nd(1.0e-15 / u.second, scaling)

for material in materials:
    rheology = material.rheology
    if rheology.cohesion != None and rheology.friction != None:
        C = rheology.cohesion
        Phi = rheology.friction
        YieldStress = C*fn.math.cos(Phi) + pressureField * fn.math.sin(Phi)
        rheology.plasticity =  0.5 * YieldStress / fn.misc.max(strainRate_2ndInvariantFn, DefaultSRInvariant) 
    else:
        rheology.plasticity = rheology.viscosity

### Viscosity Limiter

In [ ]:
# Viscosity limiter
min_viscosity = air.rheology.viscosity
max_viscosity = nonDimensionalize(1e25 * u.pascal * u.second, scaling)

ViscosityMap = {}
for material in materials:
    plasticity = material.rheology.plasticity
    viscosity = material.rheology.viscosity
    ViscosityMap[material.index] = fn.exception.SafeMaths( fn.misc.min(
                                         fn.misc.max(
                                            fn.misc.min(plasticity, viscosity), 
                                            min_viscosity),
                                      max_viscosity))

viscosityFn = fn.branching.map(fn_key = materialIndexField, mapping = ViscosityMap)

BGViscosityMap = {}
for material in materials:
    BGViscosityMap[material.index] = fn.misc.max(fn.misc.min(material.rheology.viscosity, max_viscosity), min_viscosity)

backgroundViscosityFn = fn.branching.map(fn_key = materialIndexField, mapping = BGViscosityMap)

### Yielding criterion

In [ ]:
SYconditions = [(viscosityFn < backgroundViscosityFn, strainRate_2ndInvariantFn),
                (True, 0.0)]

isYielding = fn.branching.conditional(SYconditions)

## Initial Temperature

Calculate Steady state thermal field as initial condition

In [ ]:
surfaceTemp = nd(surfaceTemp, scaling)
baseModelTemp = nd(baseModelTemp, scaling)

In [ ]:
DiffusivityMap = {}
for material in materials:
    DiffusivityMap[material.index] = material.diffusivity

DiffusivityFn = fn.branching.map(fn_key = materialIndexField, mapping = DiffusivityMap)

In [ ]:
HeatProdMap = {}
for material in materials:
    HeatProdMap[material.index] = material.radiogenicHeatProd / (material.density * material.capacity)

HeatProdFn = fn.branching.map(fn_key = materialIndexField, mapping = HeatProdMap)

In [ ]:
base   = mesh.specialSets["MinK_VertexSet"]

coords = fn.input()
airnodes = []

for index, coords in enumerate(mesh.data):
    if coords[2] > crust.top:
        temperatureField.data[index] = surfaceTemp
        airnodes.append(index)

temperatureField.data[base.data] = baseModelTemp       
        
airIndices = uw.mesh.FeMesh_IndexSet(mesh, topologicalIndex=0, size=mesh.nodesGlobal, fromObject=airnodes)

In [ ]:
temperatureBCs0 = uw.conditions.DirichletCondition( variable = temperatureField, 
                                                    indexSetsPerDof = (airIndices+base,))

heatequation = uw.systems.SteadyStateHeat(temperatureField=temperatureField,
                                          fn_diffusivity=DiffusivityFn,
                                          fn_heating = HeatProdFn,
                                          conditions=[temperatureBCs0,])
heatsolver = uw.systems.Solver(heatequation)
heatsolver.solve(nonLinearIterate=True)

In [ ]:
checkpoint_number = 0
time_year = 0.
mH = mesh.save(os.path.join(uw_output_path, "mesh.h5"), scaling, units=u.kilometers)
file_prefix = os.path.join(uw_output_path, 'temperature-%s' % checkpoint_number)
handle = temperatureField.save('%s.h5' % file_prefix, scaling=scaling, units=u.degK)
temperatureField.xdmf('%s.xdmf' % file_prefix, handle, 'temperature', mH, 'mesh', modeltime=time_year)

## Initial Pressure

Calculate lithostatic pressure field as initial condition.

In [ ]:
def smooth_pressure(mesh, pressure):
    # Smooths the pressure field.
    # Assuming that pressure lies on the submesh, do a cell -> nodes -> cell
    # projection.
    NodePressure = uw.mesh.MeshVariable(mesh, nodeDofCount=1)
    Cell2Nodes = uw.utils.MeshVariable_Projection(NodePressure, pressure, type=0)
    #Nodes2Cell = uw.utils.MeshVariable_Projection(pressure, NodePressure, type=0)
    Cell2Nodes.solve()
    #Nodes2Cell.solve()

In [ ]:
pressureField.data[:], LPresBot = lithoPressure(mesh,densityFn, gravity)
smooth_pressure(mesh, pressureField)

In [ ]:
checkpoint_number = 0
time_year = 0.
mH = mesh.save(os.path.join(uw_output_path, "mesh.h5"), scaling, units=u.kilometers)
file_prefix = os.path.join(uw_output_path, 'pressure-%s' % checkpoint_number)
handle = pressureField.save('%s.h5' % file_prefix, scaling=scaling, units=u.megapascal)
pressureField.xdmf('%s.xdmf' % file_prefix, handle, 'pressure', mH, 'mesh', modeltime=time_year)    

## Initial Stress Field

In [ ]:
if BottomStressBC:
    val = LPresBot[4]

    for node in mesh.specialSets["MinK_VertexSet"]:
        stressField.data[node] = [0.0, -1.0*val, 0.0]

# Boundary conditions

In [ ]:
left  = mesh.specialSets["MinI_VertexSet"] 
right = mesh.specialSets["MaxI_VertexSet"]
front = mesh.specialSets["MinJ_VertexSet"]
back  = mesh.specialSets["MaxJ_VertexSet"]

Bottom = mesh.specialSets["MinK_VertexSet"]
Top = mesh.specialSets["MaxK_VertexSet"]

## Kinematic Boundary Conditions

In [ ]:
half_rate = nd(half_rate, scaling)

for index in left:
    velocityField.data[index] = [half_rate,0.0,0.0]
for index in right:
    velocityField.data[index] = [-1.0*half_rate,0.0,0.0]

In [ ]:
if BottomStressBC:
    stokesBC = uw.conditions.DirichletCondition(variable=velocityField, 
                                                indexSetsPerDof=(iWalls, topCorners+botCorners))

    stressBc = uw.conditions.NeumannCondition(flux=stressField, variable=velocityField, 
                                              nodeIndexSet=base-botCorners)
    
else:
    stokesBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                                 indexSetsPerDof = (left+right, Bottom))

    stressBc = None

## Thermal Boundary Conditions

In [ ]:
DirichletTBCs = uw.conditions.DirichletCondition( variable = temperatureField, indexSetsPerDof = [airIndices+Bottom,])

advdiffSystem = uw.systems.AdvectionDiffusion(temperatureField,
                                              temperatureDotField,
                                              velocityField=velocityField,
                                              fn_diffusivity=DiffusivityFn,
                                              fn_sourceTerm=HeatProdFn,
                                              conditions=[DirichletTBCs])

## Stokes

In [ ]:
if BottomStressBC:
    conditions = [stokesBC, stressBc]
else:
    conditions = [stokesBC,]


stokes = uw.systems.Stokes(    velocityField = velocityField, 
                               pressureField = solverPressure,
                               conditions    = conditions,
                               fn_viscosity  = viscosityFn, 
                               fn_bodyforce  = buoyancyFn)

solver = uw.systems.Solver( stokes )
solver.set_inner_method("mumps")
solver.set_penalty(1.0e6)

## Initial Damage Zone

In [ ]:
def damage(xx, yy, Lx, Ly):
    return (1 - np.cos(2.0 * np.pi * xx / Lx))**4 * (1-np.cos(2.0*np.pi * yy / Ly)) 

for id, (x,y) in enumerate(swarm.particleCoordinates.data):
    if materialIndexField.data[id] != air.index:
        cumulativeTotalStrain.data[id] = damage(x, y, (maxX-minX), (crust.top-minY))
    
fn_minmax = fn.view.min_max(cumulativeTotalStrain)
fn_minmax.evaluate(swarm)
cumulativeTotalStrain.data[...] = cumulativeTotalStrain.data[...] / fn_minmax.max_global() * 1.75
cumulativeTotalStrain.data[...] *= np.random.rand(*cumulativeTotalStrain.data.shape[:])

In [ ]:
sH = swarm.save(os.path.join(uw_output_path, 'swarm-%s.h5' % checkpoint_number), scaling, units=u.kilometers)
file_prefix = os.path.join(uw_output_path, 'cumulativeTotalStrain-%s' % checkpoint_number)
handle = cumulativeTotalStrain.save('%s.h5' % file_prefix)
cumulativeTotalStrain.xdmf('%s.xdmf' % file_prefix, handle, 'cumulativeTotalStrain', sH, 'swarm', modeltime=time_years)

## Simple Erosion

In [ ]:
sealevel = nd(0.0 * u.kilometer, scaling)

AboveWaterCond = [(((materialIndexField > air.index + 0.1 ) & (fn.input()[1] > sealevel)), air.index), (True, materialIndexField)]
erosionFn = fn.branching.conditional(AboveWaterCond)

## Simple Sedimentation

In [ ]:
UnderWaterCond = [(((materialIndexField < air.index + 0.1 ) & (fn.input()[1] < sealevel)), sediment.index), (True, materialIndexField)]
sedimentationFn = fn.branching.conditional(UnderWaterCond)

# Plastic strain accumulation Envelope

In [ ]:
import math
# envelope to ensure the localisation stays clear of the boundary
def boundary(xx, minX, maxX, width, power):
    zz = xx / (maxX - minX)
    return (np.tanh(zz*width) + np.tanh((1.0-zz)*width) - math.tanh(width))**power

# Passive Tracers

In [ ]:
xp = np.linspace(minX, maxX, 100)
yp = np.linspace(minY, maxY, 100)

xp, yp = np.meshgrid(xp, yp)
xp = xp.flatten()
yp = yp.flatten()
zp = np.zeros(xp.shape)

coordsTracers = zip(xp,yp,zp)

surfaceSwarm = uw.swarm.Swarm( mesh = mesh, particleEscape=True)
surfaceSwarm.add_particles_with_coordinates(np.array(coordsTracers))
surfaceSwarmAdvector = uw.systems.SwarmAdvector( swarm=surfaceSwarm, velocityField=velocityField, order=2 )

surfaceSwarmElevation = uw.swarm.SwarmVariable(swarm=surfaceSwarm, dataType="double", count=surfaceSwarm.particleCoordinates.data[:,1].size)
surfaceSwarmElevation.data[...] = surfaceSwarm.particleCoordinates.data[:,1]

pass

In [ ]:
sH = surfaceSwarm.save(os.path.join(uw_output_path, 'SurfaceSwarm-%s.h5' % checkpoint_number), scaling, units=u.kilometers)
file_prefix = os.path.join(uw_output_path, 'SurfaceTracers-%s' % checkpoint_number)
surfaceSwarmElevation.xdmf('%s.xdmf' % file_prefix, handle, 'SurfaceTracers', sH, 'SurfaceSwarm', modeltime=time_years)

# Pressure Calibration

In [ ]:
surfaceArea = uw.utils.Integral(fn=1.0,mesh=mesh, integrationType='surface', surfaceIndexSet=top)
surfacePressureIntegral = uw.utils.Integral(fn=solverPressure, mesh=mesh, integrationType='surface', surfaceIndexSet=top)

# Obtain V,P and remove null-space / drift in pressure
def nonLinearSolver(step, nl_tol=1e-2, nl_maxIts=20):
    # a hand written non linear loop for stokes, with pressure correction
    
    er = 1.0
    its = 0                      # iteration count
    v_old = velocityField.copy() # old velocityField 
    residuals = []

    while er > nl_tol and its < nl_maxIts:

        v_old.data[:] = velocityField.data[:]
        solver.solve(nonLinearIterate=False)

        # pressure correction
        (area,) = surfaceArea.evaluate()
        (p0,) = surfacePressureIntegral.evaluate() 
        pressureField.data[:] = solverPressure.data[:] - (p0 / area)
        smooth_pressure(mesh, pressureField)

        # calculate relative error
        absErr = uw.utils._nps_2norm(velocityField.data-v_old.data)
        magT   = uw.utils._nps_2norm(v_old.data)
        er = absErr/magT
        residuals.append(er)

        if uw.rank() == 0:
            if not is_kernel():
                print(er)
                
        its += 1
        
    del(v_old)

## Checkpoint Function

In [ ]:
def checkpoint_function(checkpoint_number, time_years):
    
    print('Writing Underworld outputs (years)',time_years)
    
    mH = mesh.save(os.path.join(outputDir, "mesh.h5"), scaling, units=u.kilometers)

    file_prefix = os.path.join(outputDir, 'velocity-%s' % checkpoint_number)
    handle = velocityField.save('%s.h5' % file_prefix, scaling=scaling, units=u.centimeter/u.year)
    velocityField.xdmf('%s.xdmf' % file_prefix, handle, 'velocity', mH, 'mesh', modeltime=time_years)

    file_prefix = os.path.join(outputDir, 'pressure-%s' % checkpoint_number)
    handle = pressureField.save('%s.h5' % file_prefix, scaling=scaling, units=u.megapascal)
    pressureField.xdmf('%s.xdmf' % file_prefix, handle, 'pressure', mH, 'mesh', modeltime=time_years)
    
    file_prefix = os.path.join(outputDir, 'temperature-%s' % checkpoint_number)
    handle = temperatureField.save('%s.h5' % file_prefix, scaling=scaling, units=u.degC)
    temperatureField.xdmf('%s.xdmf' % file_prefix, handle, 'temperature', mH, 'mesh', modeltime=time_years)

    sH = swarm.save(os.path.join(outputDir, 'swarm-%s.h5' % checkpoint_number), scaling, units=u.kilometers)

    file_prefix = os.path.join(outputDir, 'material-%s' % checkpoint_number)
    handle = materialIndexField.save('%s.h5' % file_prefix)
    materialIndexField.xdmf('%s.xdmf' % file_prefix, handle, 'material', sH, 'swarm', modeltime=time_years)
   
    file_prefix = os.path.join(outputDir, 'Pstrain-%s' % checkpoint_number)
    handle = cumulativeTotalStrain.save('%s.h5' % file_prefix)
    cumulativeTotalStrain.xdmf('%s.xdmf' % file_prefix, handle, 'Pstrain', sH, 'swarm', modeltime=time_years)
    
    strainRate.data[...] = strainRate_2ndInvariantFn.evaluate(swarm)
    file_prefix = os.path.join(outputDir, 'strainR-%s' % checkpoint_number)
    handle = strainRate.save('%s.h5' % file_prefix, scaling, units= 1.0 / u.seconds)
    strainRate.xdmf('%s.xdmf' % file_prefix, handle, 'strainR', sH, 'swarm', modeltime=time_years)

    tsH = surfaceSwarm.save(os.path.join(outputDir, 'tracer_swarm-%s.h5' % checkpoint_number),
                            scaling=scaling, units=u.kilometers)

    file_prefix = os.path.join(outputDir, 'tracer-%s' % checkpoint_number)
    handle = surfaceSwarm.particleCoordinates.save('%s.h5' % file_prefix, scaling=scaling, units=u.kilometers)
    surfaceSwarm.particleCoordinates.xdmf('%s.xdmf' % file_prefix, handle, 'tvar', tsH, 'tracer', modeltime=time_years)

# Main simulation loop
----------------------------------------

In [ ]:
advector   = uw.systems.SwarmAdvector( swarm = swarm, velocityField=velocityField, order=2 )
advectorSurface  = uw.systems.SwarmAdvector( swarm = surfaceSwarm, velocityField=velocityField, order=2 )

population_control = uw.swarm.PopulationControl(swarm, 
                                                aggressive=True,splitThreshold=0.15, 
                                                maxDeletions=2,maxSplits=10,
                                                particlesPerCell=20)

In [ ]:
time = 0.
endTime = nd(5.0 * u.megayear, scaling)

step = 0
nsteps = 30
checkpoint_number = 0

def get_dt(CFL=0.2, threshold = 2500):
    dt3 = nd(threshold * u.year, scaling)
    dt4 = CFL * advdiffSystem.get_max_dt()
    return min(dt3,dt4)


while step < nsteps:
    
    nonLinearSolver(step, nl_tol=1e-2, nl_maxIts=40)
    
    if step % 10 == 0:
        checkpoint_number +=1
        checkpoint_function(checkpoint_number, Dimensionalize(time, scaling, u.years).magnitude)

    # obtain a timestep and apply a courant factor
    dt = get_dt()
         
    if uw.rank()==0:   
        print('step = {0:6d}; time = {1:.3e}'.format(step, Dimensionalize(time, scaling, u.megayears)))
        
    uw.barrier()
    
    # update plastic strain
    plasticStrainIncrement = dt * isYielding.evaluate(swarm)
    weight = boundary(swarm.particleCoordinates.data[:,0], minX, maxX, 20, 4)
    plasticStrainIncrement[:,0] *= weight
    cumulativeTotalStrain.data[:] += plasticStrainIncrement
    
    # Solve for temperature
    advdiffSystem.integrate(dt)

    # integrate swarms in time
    advector.integrate(dt, update_owners=True)  
    advectorSurface.integrate(dt, update_owners=True)  

    # repopulate swarm.
    population_control.repopulate() 
        
    # Do basic surface processes
    if Sedimentation: 
        materialIndexField.data[:] = sedimentationFn.evaluate(swarm)
    
    if Erosion:
        materialIndexField.data[:] = erosionFn.evaluate(swarm)
       
    step +=1
    time +=dt

In [ ]:
if is_kernel():
    FigMaterials.show()

In [ ]:
if is_kernel():
    FigVelocity.show()

In [ ]:
if is_kernel():
    FigPlasticStrain.show()